In [1]:
import glob
import pandas as pd
import math
from sklearn.neighbors import BallTree
import numpy as np

In [2]:
# Load GHGSat plume csvs

GHGS_paths = glob.glob('C:/Users/hughr/OneDrive/Documents/RMI/GHGSat/OneDrive_1_5-30-2023/*.csv')
GHGS = []
for p in GHGS_paths:
    df = pd.read_csv(p)
    # Cut off a bunch of empty columns
    df = df.loc[:, 'Observation ID':'Source Rate Error [%]']
    GHGS.append(df)

GHGS = pd.concat(GHGS, axis=0)
GHGS.reset_index(drop=True, inplace=True)
GHGS

,Observation ID,Plume ID,Acquisition Date [UTC],Plume Latitude,Plume Longitude,Source Rate [kg/hr],Source Rate Error [%]
0,7RXXj_Z,7RXXj_Z-1,1/1/2021 11:10,40.26301,-3.63573,966.58,32.04
1,5RXcmGQ,5RXcmGQ-1,1/3/2021 7:30,43.53340,52.15465,520.91,46.61
2,5Rb2sl7,5Rb2sl7-1,1/5/2021 18:07,36.79302,-108.38876,2136.99,29.62
3,6Rc32C6,6Rc32C6-1,1/7/2021 3:22,36.25738,112.92264,6954.02,26.35
4,6Rc32C6,6Rc32C6-2,1/7/2021 3:22,36.23306,112.94636,9495.94,25.96
...,...,...,...,...,...,...,...
5342,LaU72-W,LaU72-W--1,09/29/2022 18:26,-2.07063,-79.96116,2635.08,67.32
5343,DaU7LoR,DaU7LoR--1,09/29/2022 20:14,31.73300,-102.17606,316.62,41.04
5344,-aV1AFV,NaN,09/30/2022 08:53,54.88000,15.41000,NaN,NaN
5345,6aV3fFV,NaN,09/30/2022 10:28,54.88000,15.41000,79000.00,29.11


In [3]:
# Sort by plume size
#GHGS = GHGS.sort_values('Source Rate [kg/hr]', ascending=False).reset_index(drop=True)

In [4]:
# Load Climate TRACE asset database

TRACE = pd.read_csv('C:/Users/hughr/OneDrive/Documents/RMI/Climate_TRACE/waste/asset_solid-waste-disposal_emissions.csv')
TRACE = TRACE[TRACE['gas'] == 'ch4'].reset_index(drop=True)
TRACE

,asset_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,capacity,capacity_units,capacity_factor,activity,activity_units,created_date,modified_date,asset_name,asset_type,st_astext
0,1835960681,KOR,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,228071.908800,0.034650,NaN,1.144865e+08,landfill_area_sq_meters,NaN,6.582168e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Sudokwon,Sanitary Landfills,POINT(126.613998413085 37.5772357911111)
1,1835960682,TUR,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,131856.520200,0.030742,NaN,7.534171e+07,landfill_area_sq_meters,NaN,4.289130e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Odayeri,Sanitary Landfills,POINT(28.8581657 41.2161766)
2,1835960683,CHN,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,131104.987200,0.034650,NaN,6.413101e+07,landfill_area_sq_meters,NaN,3.783697e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Laogang Landfill,Sanitary Landfill,POINT(121.874575 31.056294)
3,1835960821,USA,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,9868.230000,0.006824,NaN,3.108446e+07,landfill_area_sq_meters,NaN,1.446208e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,King George Landfill Inc.,Sanitary Landfills,POINT(-77.30596 38.26964)
4,1835960684,CHN,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,104839.594100,0.028723,NaN,5.663107e+07,landfill_area_sq_meters,NaN,3.650000e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Jiangcungou Landfill,Sanitary Landfill,POINT(109.100563 34.245692)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4740,1835965296,KHM,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,29.341661,0.017325,NaN,3.870912e+04,landfill_area_sq_meters,NaN,1.693604e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Udong, Odongk, Kampong Speu, 855, Cambodia",Dumpsites,POINT(104.71550431136072 11.808565128092692)
4741,1835965297,PHL,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,29.301214,0.017325,NaN,3.865576e+04,landfill_area_sq_meters,NaN,1.691269e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Bangkal, Baraka, Bulacan, Central Luzon, 3013,...",Dumpsites,POINT(121.08773982254641 14.916148899405837)
4742,1835965298,KHM,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,28.380501,0.017325,NaN,3.744110e+04,landfill_area_sq_meters,NaN,1.638126e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Doun Kaev, Takeo, 885, Cambodia",Dumpsites,POINT(104.68932165937792 10.973150260323228)
4743,1835965299,NGA,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,28.290846,0.012097,NaN,3.430273e+04,landfill_area_sq_meters,NaN,2.338728e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Tarkwa Bay, Amuwo Odofin, Lagos, 71510, Nigeria",Dumpsites,POINT(3.392517451014247 6.406222836614646)


In [5]:
# Define a function to split st_astext column into lat and lon columns
def split_point(point):
    lon = float(point.split(' ')[0].split('(')[-1])
    lat = float(point.split(' ')[1].split(')')[0])
    return pd.Series((lat, lon))

TRACE[['lat', 'lon']] = TRACE['st_astext'].apply(split_point)

In [6]:
# Get just the methane
TRACE = TRACE[TRACE['gas'] == 'ch4']

In [7]:
# Sort by landfill size, so smaller facilities will be dropped if they are near larger ones
#TRACE_sorted = TRACE.sort_values('activity', ascending=False).reset_index(drop=True)

In [8]:
# # Drop points TRACE sites within 5km of another site

# # Convert Lat/Lon to radians for use in haversine formula
# TRACE_rad = np.deg2rad(TRACE_sorted[['lat', 'lon']])

# # Construct BallTree
# TRACE_tree = BallTree(TRACE_rad, metric='haversine')

# # Initialize array to keep track of whether row should be kept
# keep = np.ones(TRACE_sorted.shape[0], dtype=bool)

# # Convert min_distance (km) to radians
# # Earth's radius is approximately 6371 km
# min_distance = 5  # distance in km
# min_distance_rad = min_distance / 6371

# # Iterate over each point
# for i, row in enumerate(TRACE_rad.values):
#     if keep[i]:  # if this point hasn't been marked for removal
#         # Find all points within min_distance of this point
#         inds = TRACE_tree.query_radius([row], r=min_distance_rad)[0]
        
#         # Mark all *other* points as not to keep
#         inds = inds[inds != i]  # remove self from list
#         keep[inds] = False

# # Filter DataFrame
# print(TRACE_sorted.shape)
# TRACE = TRACE_sorted[keep].reset_index(drop=True)
# print(TRACE.shape)

In [9]:
# Now find GHGSat plumes within 1 km of a TRACE site

# We have to remake these because we dropped points
# Convert Lat/Lon to radians for use in haversine formula
GHGS_rad = np.deg2rad(GHGS[['Plume Latitude', 'Plume Longitude']])
TRACE_rad = np.deg2rad(TRACE[['lat', 'lon']])

# Construct BallTree
TRACE_tree = BallTree(TRACE_rad, metric='haversine')

# Query the TRACE tree for GHGS points within 10 km
threshold_distance = 1 # Unit is km
indices, distances = TRACE_tree.query_radius(GHGS_rad, r=threshold_distance/6371, return_distance=True)

# Iterate through the GHGSat plume locations
matches = []
for i, (distance, index) in enumerate(zip(distances, indices)):
    # Iterate over the matches for each plume
    for d, idx in zip(distance, index):
        matches.append({
            'lat_GHGS': GHGS.at[i, 'Plume Latitude'],
            'lon_GHGS': GHGS.at[i, 'Plume Longitude'],
            'idx_GHGS': i,
            'lat_TRACE': TRACE.at[idx, 'lat'],
            'lon_TRACE': TRACE.at[idx, 'lon'],
            'idx_TRACE': idx,
            'distance': d * 6371,  # Convert radians to km
            'rate': GHGS.at[i, 'Source Rate [kg/hr]']
        })

matches_df = pd.DataFrame(matches)

In [10]:
# Do stuff with the matches if you want, drop some duplicates or etc.
matches_df

,lat_GHGS,lon_GHGS,idx_GHGS,lat_TRACE,lon_TRACE,idx_TRACE,distance,rate
0,25.02583,67.03766,10,25.028194,67.033929,1180,0.458705,1080.95
1,8.97812,38.71185,38,8.975500,38.712087,367,0.292438,1889.41
2,28.62115,77.32863,42,28.624698,77.327405,154,0.412240,1497.40
3,14.14936,-87.22384,50,14.148431,-87.224144,370,0.108376,500.10
4,19.12596,72.95071,59,19.124909,72.943392,3246,0.777631,20148.10
...,...,...,...,...,...,...,...,...
405,43.08518,-77.37149,5248,43.086950,-77.379430,167,0.674167,771.78
406,12.95278,80.22205,5266,12.954919,80.223928,668,0.312975,1530.15
407,41.21459,28.15088,5267,41.214423,28.149789,1921,0.093130,5661.72
408,14.80479,-17.31091,5312,14.803593,-17.313252,177,0.284753,5248.04


In [11]:
GHGS.loc[10, :]

Observation ID                   7Rk4Utg
Plume ID                       7Rk4Utg-1
Acquisition Date [UTC]    1/15/2021 6:12
Plume Latitude                  25.02583
Plume Longitude                 67.03766
Source Rate [kg/hr]              1080.95
Source Rate Error [%]              68.98
Name: 10, dtype: object

In [12]:
TRACE.loc[1180, :]

asset_id                                          1835961831
iso3_country                                             PAK
original_inventory_sector               solid-waste-disposal
start_time                               2021-01-01 00:00:00
end_time                                 2021-12-31 00:00:00
temporal_granularity                                  annual
gas                                                      ch4
emissions_quantity                               2268.792631
emissions_factor                                    0.015896
emissions_factor_units                                   NaN
capacity                                         18227204.44
capacity_units                       landfill_area_sq_meters
capacity_factor                                          NaN
activity                                         142731.2789
activity_units               estimated_tonnes_waste_in_place
created_date                             2023-01-24 00:00:00
modified_date           

In [13]:
# Calculate average rate from sites that had multiple plumes
average_rate = matches_df.groupby('idx_TRACE')['rate'].mean()

# Remove all but one plume per site
print(matches_df.shape)
matches_df.drop_duplicates(subset='idx_TRACE', inplace=True, ignore_index=True)
print(matches_df.shape)

# Add the average values back in
matches_df.index = matches_df['idx_TRACE']
matches_df.loc[average_rate.index, 'avg_rate'] = average_rate

(410, 8)
(85, 8)


In [14]:
matches_df

,lat_GHGS,lon_GHGS,idx_GHGS,lat_TRACE,lon_TRACE,idx_TRACE,distance,rate,avg_rate
idx_TRACE,,,,,,,,,
1180,25.02583,67.03766,10,25.028194,67.033929,1180,0.458705,1080.95,2334.242500
367,8.97812,38.71185,38,8.975500,38.712087,367,0.292438,1889.41,1731.137500
154,28.62115,77.32863,42,28.624698,77.327405,154,0.412240,1497.40,2090.855517
370,14.14936,-87.22384,50,14.148431,-87.224144,370,0.108376,500.10,500.100000
3246,19.12596,72.95071,59,19.124909,72.943392,3246,0.777631,20148.10,8225.620000
...,...,...,...,...,...,...,...,...,...
252,25.85884,-80.35145,5239,25.858600,-80.350800,252,0.070302,926.63,926.630000
209,43.21118,-78.97387,5241,43.210708,-78.978997,209,0.418809,748.34,748.340000
167,43.08518,-77.37149,5248,43.086950,-77.379430,167,0.674167,771.78,771.780000


In [15]:
# import geopandas as gpd
# import matplotlib.pyplot as plt
# from shapely.geometry import Point

# matches_df['Coordinates'] = list(zip(matches_df.lon_TRACE, matches_df.lat_TRACE))
# matches_df['Coordinates'] = matches_df['Coordinates'].apply(Point)

# gdf = gpd.GeoDataFrame(matches_df, geometry='Coordinates')

# # Plotting
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# fig, ax = plt.subplots(figsize = (16,6))
# fig.suptitle('TRACE sites with GHGSat plumes', fontsize = 'xx-large', fontweight = 'bold')

# world.boundary.plot(ax=ax, color='black', linewidth=.5)

# # Normalize the 'value' column for color mapping
# vmin = matches_df['rate'].min()
# vmax = matches_df['rate'].max()
# cmap = plt.get_cmap('Reds')
# gdf['color'] = gdf['rate'].apply(lambda x: cmap((x - vmin) / (vmax - vmin)))

# # Plot points with color and size based on 'value'
# #gdf.plot(marker='o', color=gdf['color'], markersize=gdf['rate']/10, ax=ax)
# gdf.plot(marker='o', markersize=gdf['rate']/20, ax=ax)

# plt.show()

# # To save the figure
# #fig.savefig('path_to_save.png', dpi=600)

In [16]:
matches_df.reset_index(inplace=True, drop=True)

In [17]:
from qgis.core import QgsApplication, QgsRasterLayer, QgsProject, QgsVectorLayer
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

# Create a column of shapely points for geometry
matches_df['Coordinates'] = list(zip(matches_df.lon_TRACE, matches_df.lat_TRACE))
matches_df['Coordinates'] = matches_df['Coordinates'].apply(Point)
gdf = gpd.GeoDataFrame(matches_df, geometry='Coordinates')

# Supply path to qgis install location
QgsApplication.setPrefixPath("C:/OSGeo4W/apps/qgis-ltr", True)

# Create a reference to the QgsApplication. Setting the second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

# Create a new QGIS project
project = QgsProject.instance()

# Set the initial CRS to WGS84 (lat/long degrees)
gdf.set_crs("EPSG:4326", inplace=True)

# Reproject to the CRS of the basemap (Web Mercator)
gdf = gdf.to_crs("EPSG:3857")

# Save the GeoDataFrame to a shapefile
shapefile_path = os.path.join('C:/Users/hughr/OneDrive/Documents/RMI/GHGSat', 'points.shp')
gdf.to_file(shapefile_path)

# Add the points shapefile to the project
point_layer = QgsVectorLayer(shapefile_path, 'Points', 'ogr')

if not point_layer.isValid():
    print('Layer failed to load.')
else:
    project.addMapLayer(point_layer)

# Save the project to a .qgs file
project.write('C:/Users/hughr/OneDrive/Documents/RMI/GHGSat/GHGSat_test.qgs')

# Close the QgsApplication. This will remove any temporary files and cleanup 
qgs.exitQgis()

C:\Users\hughr\AppData\Local\Temp\ipykernel_31552\1554638970.py:32: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(shapefile_path)


In [18]:
matches_df

,lat_GHGS,lon_GHGS,idx_GHGS,lat_TRACE,lon_TRACE,idx_TRACE,distance,rate,avg_rate,Coordinates
0,25.02583,67.03766,10,25.028194,67.033929,1180,0.458705,1080.95,2334.242500,POINT (67.033929 25.028194)
1,8.97812,38.71185,38,8.975500,38.712087,367,0.292438,1889.41,1731.137500,POINT (38.71208667 8.975500449)
2,28.62115,77.32863,42,28.624698,77.327405,154,0.412240,1497.40,2090.855517,POINT (77.327405 28.624698)
3,14.14936,-87.22384,50,14.148431,-87.224144,370,0.108376,500.10,500.100000,POINT (-87.224144 14.148431)
4,19.12596,72.95071,59,19.124909,72.943392,3246,0.777631,20148.10,8225.620000,POINT (72.94339202947502 19.12490946633295)
...,...,...,...,...,...,...,...,...,...,...
80,25.85884,-80.35145,5239,25.858600,-80.350800,252,0.070302,926.63,926.630000,POINT (-80.3508 25.8586)
81,43.21118,-78.97387,5241,43.210708,-78.978997,209,0.418809,748.34,748.340000,POINT (-78.978997 43.210708)
82,43.08518,-77.37149,5248,43.086950,-77.379430,167,0.674167,771.78,771.780000,POINT (-77.37943 43.08695)
83,12.95278,80.22205,5266,12.954919,80.223928,668,0.312975,1530.15,1530.150000,POINT (80.2239275 12.95491878)
